In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy import stats
from scipy.optimize import fsolve

In [47]:
def system(x, T, L, r, k, E, sigma_E, t=0):
    sigma, v = x
    d1 = (np.log(v / L) + ((r - k) + sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t))
    d2 = (np.log(v / L) + ((r - k) - sigma ** 2 / 2) * (T - t)) / (sigma * np.sqrt(T - t))
    return v * stats.norm.cdf(d1) - L * np.exp(-r * T) * stats.norm.cdf(d2) - E, \
           sigma_E * E - stats.norm.cdf(d1) * sigma * v


def insolvency_probability(T, parameters, t=0):
    solution = fsolve(
        lambda x: system(x,
                         T,
                         parameters["L"],
                         parameters["r"],
                         parameters["k"],
                         parameters["E"],
                         parameters["sigma_E"]),
        x0=np.array([0.01, parameters["market_cap"]]))
    sigma, v = solution[0], solution[1]
    return stats.norm.cdf(
        (np.log(parameters["L"] / v) - ((parameters["r"] - parameters["k"]) - sigma ** 2 / 2) * (T - t)) / (
                    sigma * np.sqrt(T - t)))


C:\Users\mlubn\AppData\Local\Temp\ipykernel_27004\2883816569.py:10: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  solution = fsolve(


0.9999999619669017

In [ ]:
all_stock_data = yf.download(['WMT', 'CCC.WA'], start='2023-06-12', end='2024-06-13')
raw_close = pd.DataFrame({'ccc': all_stock_data.Close['CCC.WA'], 'wmt': all_stock_data.Close.WMT})
close = raw_close.interpolate("linear")
log_std = (np.log(close) - np.log(close.shift(1))).std()
log_std
wmt = yf.Ticker("WMT")

In [ ]:
wmt_params = {"L": wmt.info["totalDebt"],
              "r": (4.34 / 100 + 1) ** (1 / 250) - 1,
              "k": (wmt.info["dividendYield"] + 1) ** (1 / 250) - 1,
              "sigma_E": log_std.wmt,
              "E": wmt.info["currentPrice"],
              "market_cap": wmt.info['marketCap']}
ccc = yf.Ticker("CCC.WA")

In [ ]:
ccc_params = {"L": ccc.info["totalDebt"],
              "r": (5.75 / 100 + 1) ** (1 / 250) - 1,
              "k": 0,
              "sigma_E": log_std.ccc,
              "E": ccc.info["currentPrice"],
              "market_cap": ccc.info['marketCap']}


In [ ]:
insolvency_probability(250, wmt_params)

In [ ]:
insolvency_probability(250, ccc_params)